In [1]:
import os

In [2]:
os.chdir('C:/Users/op/Desktop/assign/')

In [3]:
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
import collections
import numpy as np

from sklearn.utils.class_weight import compute_class_weight
import pandas as pd

In [5]:
import matplotlib.pyplot as plt
from torchvision import transforms, datasets
from torch.utils.data import (
    Dataset,
    DataLoader,
)
import torch
import glob
from torchvision.datasets import ImageFolder
from torchvision.transforms import (
    RandomResizedCrop,
    RandomHorizontalFlip,
    ColorJitter,
    RandomGrayscale,
    RandomApply,
    Compose,
    GaussianBlur,
    ToTensor,
)
from skimage import io

In [6]:
img_path   = './data/jpg/'
label_path = './data/imagelabels.mat'

In [7]:
## check class distribution in the data
labels = loadmat('./data/imagelabels.mat')['labels'][0]
label_distribution = collections.Counter(labels)

In [9]:
label_freq = []
_label = []
for lbl, freq in label_distribution.items():
    label_freq.append(freq)
    _label.append(lbl)

_label = np.array(_label)
label_freq = np.array(label_freq)
sort_index = label_freq.argsort()
label_freq.sort()
_label = _label[sort_index]

### check for class imbalance
print('Class with highest frequency: {0}, {1}'.format(_label[-1], label_freq[-1]))
print('Class with lowest frequency: {0}, {1}'.format(_label[0], label_freq[0]))
print('Class imbalance : {}'.format(label_freq[-1]/label_freq[0]))

Class with highest frequency: 51, 258
Class with lowest frequency: 34, 40
Class imbalance : 6.45


In [10]:
### Test-train split
X_train_index, X_test_index, y_train, y_test = train_test_split([i for i in range(0,len(labels))], labels, stratify=labels, 
                                                    test_size=0.8, shuffle=True)
files = np.array(glob.glob("./data/jpg/**.jpg"))
dataset_train_files = files[X_train_index]
dataset_test_files  = files[X_test_index]

In [11]:
### The issue of class imbalance can be mitigated with a weighted cross-entropy loss function whereby each class is weighted
# in proportion to its imbalance
class_weights = compute_class_weight(class_weight = "balanced", classes= np.unique(labels), y= labels)
labels_dict = {'class': _label, 'frequency':label_freq, 'weight':class_weights[_label-1]}
label_frame = pd.DataFrame.from_dict(labels_dict)
print(label_frame)

     class  frequency    weight
0       34         40  2.007108
1        3         40  2.007108
2        7         40  2.007108
3       45         40  2.007108
4        1         40  2.007108
..     ...        ...       ...
97      89        184  0.436328
98      73        194  0.413837
99      46        196  0.409614
100     77        251  0.319858
101     51        258  0.311180

[102 rows x 3 columns]


In [8]:
labels

array([77, 77, 77, ..., 62, 62, 62], dtype=uint8)